In [2]:
import labels
import pandas as pd

from const import *

In [3]:
labels_df = labels.load_labels()

In [4]:
results = pd.read_csv("../results/crawler_results.csv")
results = results.rename(columns={"key": "filename", "value": "pred"})
results["pred"].unique()
results = results[results["pred"] != "Review"]
results["pred"] = results["pred"] == "True"

In [5]:
def points(label, pred): 
    label_str, pred_str = str(label), str(pred)
    points = LOSS[label_str][pred_str]
    return points

def max_points(label): 
    label_str, pred_str = str(label), str(label)
    points = LOSS[label_str][pred_str]
    return points

In [6]:
merged = labels_df.merge(results, left_on="filename", right_on="filename")
merged["aligned"] = (merged["sensitive"] == merged["pred"])
merged["points"] = merged.apply(lambda row: points(row["sensitive"], row["pred"]), axis = 1)
merged["max_points"] = merged.apply(lambda row: max_points(row["sensitive"]), axis = 1)
merged

,filename,sensitive,name,extension,pred,aligned,points,max_points
0,within-resource.xml,True,within-resource,xml,True,True,20,20
1,fly-camera-book.xml,True,fly-camera-book,xml,True,True,20,20
2,debitis.msg,True,debitis,msg,True,True,20,20
3,incidunt-officia.msg,True,incidunt-officia,msg,True,True,20,20
4,gun-expect-relate-civil-receive-be.ps1,True,gun-expect-relate-civil-receive-be,ps1,False,False,-20,20
...,...,...,...,...,...,...,...,...
750,week-we-page-almost-response-other.log,False,week-we-page-almost-response-other,log,True,False,-2,2
751,west-total-million-quality-large-hear.log,False,west-total-million-quality-large-hear,log,False,True,2,2
752,will-way-could-head-goal-cost.log,False,will-way-could-head-goal-cost,log,False,True,2,2
753,wonder-sort-need-join-low-less.log,False,wonder-sort-need-join-low-less,log,True,False,-2,2


In [7]:
perc_points = merged["points"].sum() / merged["max_points"].sum()
perc_points

0.22814321398834306

In [8]:
points_df = pd.DataFrame()
points_df["points"] = merged.groupby(by = "extension")["points"].sum()
points_df["max_points"] = merged.groupby(by = "extension")["max_points"].sum()
points_df

,points,max_points
extension,,
csv,396,396
db,112,112
docx,12,132
html,50,174
log,54,218
md,-154,366
msg,50,50
pdf,-918,1110
pem,1140,1140


In [9]:
merged[merged["extension"] == "log"]["aligned"].mean()
labels_df[labels_df["extension"] == "log"]["sensitive"].mean()

0.01

In [12]:
merged.groupby(by = "extension")["aligned"].mean().sort_values(ascending = False)

extension
csv     1.000000
db      1.000000
msg     1.000000
pem     1.000000
xlsx    1.000000
xml     1.000000
pub     0.975610
html    0.933333
py      0.900000
md      0.803030
docx    0.750000
log     0.680000
ps1     0.666667
pdf     0.457143
txt     0.200000
Name: aligned, dtype: float64